In [1]:
import os
import rootutils
rootutils.setup_root(os.getcwd(), indicator=".project-root", pythonpath=True)

PosixPath('/Users/tomeknocon/MIM/Master_Thesis/IMMUVIS-PIGVAE')

In [2]:
from torchvision import transforms
from torchvision import datasets
import torch
from src.data.components.graphs_datamodules import  SplitPatches, CellDataset, GraphDataModule
from src.data.mnist_datamodule import MNISTDataModule

In [3]:
from torchvision.datasets import MNIST
from torch.utils.data import ConcatDataset, DataLoader, Dataset, random_split

transform = transforms.Compose([
            transforms.ToTensor(), 
            transforms.Normalize((0.1307,), (0.3081,)),
            transforms.Resize((24, 24)),
            transforms.Lambda(lambda x: x.unsqueeze(0)),
            SplitPatches(4)
        ])

trainset = MNIST(
               '../data', train=True, transform=transform
            )
testset = MNIST(
               '../data', train=False, transform=transform
            )
dataset = ConcatDataset(datasets=[trainset, testset])

In [4]:
data_train, data_val, data_test = random_split(
                dataset=dataset,
                lengths=[100, 10, 69890],
                generator=torch.Generator().manual_seed(42),
            )

In [5]:
import networkx as nx

class GridGraphDataset(Dataset):
    def __init__(self,
                 dataset,
                 grid_size: int, 
                 channels, 
        ):
        self.grid_size = grid_size
        self.dataset = dataset
        self.channels = channels
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        g = nx.grid_graph((self.grid_size, self.grid_size))
        img = self.dataset[idx][0].to(torch.float32)
        target =self.dataset[idx][1]
        return (g, img, target)

In [6]:
d = GridGraphDataset(data_train, 6, [0])

In [7]:
m = MNISTDataModule('../data', [100, 10, 69890], 32, 1)

In [8]:
m.setup()
t = m.train_dataloader()

In [9]:
for el in t:
    break

In [10]:
el

DenseGraphBatch(edge_features=[0], mask=[32, 36], node_features=[32, 36, 16], properties=[32], y=[32])